In [ ]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt

import ssm_timeSeries as ts  # my self-written time series overhead
import ssm_fit               # my self-written library for state-space model fitting
import ssm_scripts

import random
from datetime import datetime     # generate random seed for 
random.seed(datetime.now())       # np.random. Once this is fixed, all 
rngSeed = random.randint(0, 1000) # other 'randomness' is also fixed

%matplotlib inline


""" Set problem size """

yDim = 200
xDim = 10
uDim = 0

T = 500000
Trial = 1


""" Generate some sensible parameters that give a learnable problem """
sfs = '/home/marcel/Desktop/Projects/Stitching/results/test_runs/LDS_setup'
npzfile = np.load(sfs+'.npz')
yIn = npzfile['arr_0'].copy()
xIn = npzfile['arr_1'].copy()
trueParameters = {}
trueParameters['A'] = npzfile['arr_2'].copy()
trueParameters['B'] = npzfile['arr_3'].copy()
trueParameters['Q'] = npzfile['arr_4'].copy()
trueParameters['mu0'] = npzfile['arr_5'].copy()
trueParameters['V0'] = npzfile['arr_6'].copy()
trueParameters['C'] = npzfile['arr_7'].copy()
trueParameters['d'] = npzfile['arr_8'].copy()
trueParameters['R'] = npzfile['arr_9'].copy()
initParameters = [None,None,None,None,None,None,None,None]
initParameters[0] = npzfile['arr_10'].copy()
initParameters[1] = npzfile['arr_11'].copy()
initParameters[2] = npzfile['arr_12'].copy()
initParameters[3] = npzfile['arr_13'].copy()
initParameters[4] = npzfile['arr_14'].copy()
initParameters[5] = npzfile['arr_15'].copy()
initParameters[6] = npzfile['arr_16'].copy()
initParameters[7] = npzfile['arr_17'].copy()

del npzfile

yIn = yIn[:,range(T),:]
xIn = xIn[:,range(T),:]

""" Set observation protocol """

#subpops = [list(range(0,yDim)), list(range(0,yDim))]
#obsTime = [int(T/2), T]
#obsPops = [1,0]

#subpops = [[], list(range(0,65)), list(range(55,120)), list(range(115,180)), list(range(175,yDim))]
#obsPops = []
#obsTime = []

#tempStitchOrder = 3
#for t in range(1,int(T/4)+1):
#    obsTime.append(t)
#    if np.mod(t,tempStitchOrder)==0:
#        obsPops.append(1)
#    else:
#        obsPops.append(0)
#tempStitchOrder = 3
#for t in range(int(T/4)+2,int(T/2)+1):
#    obsTime.append(t)
#    if np.mod(t,tempStitchOrder)==0:
#        obsPops.append(2)
#    else:
#        obsPops.append(0)   
#for t in range(int(T/2)+2,int(3*T/4)+1):
#    obsTime.append(t)
#    if np.mod(t,tempStitchOrder)==0:
#        obsPops.append(3)
#    else:
#        obsPops.append(0)   
#obsTime.append(T)
#obsPops.append(4)
        
subpops = [[], list(range(0,yDim))]
obsPops = []
obsTime = []

tempStitchOrder = 8.888888888
obsPops = [1]
obsTime = [1] # population observed at t = 0
i = 1;
t = 0
while t < T:
    t = int(tempStitchOrder * i)
    obsPops.append(0)   # observe empty subpop
    obsTime.append(t)   # until round(i*tempStitchOrder)
    obsPops.append(1)   # then observe full subpop
    obsTime.append(t+1) # for 1 more time step
    
    i += 1
# ended while loop because last entry was >= T !    
while obsTime[-1] > T:
    obsTime.pop()
obsTime[-1] = T
    
    
obsScheme = {'subpops': subpops,
             'obsTime': obsTime,
             'obsPops': obsPops}

fitOptions = {'ifUseB' : False,  
              'maxIter': 300, 
              'ifPlotProgress' : True,
              'covConvEps' : 0,
              'ifTraceParamHist': False
             }

""" Fit the model, save results """

sf = '/home/marcel/Desktop/Projects/Stitching/results/test_runs/LDS_save_tempStitching'
[yOut,xOut,u,learnedPars,initPars,truePars] = ssm_scripts.run(xDim, yDim, uDim, T, 
                                                              obsScheme, fitOptions,
                                                              y = yIn, x = xIn,
                                                              truePars=trueParameters,
                                                              initPars=initParameters,
                                                              saveFile=sf)



In [ ]:
[A,B,Q,mu0,V0,C,d,R] = learnedPars 
Pi    = np.array([sp.linalg.solve_discrete_lyapunov(A, Q)])[0,:,:]

plt.figure(1,figsize=(15,15))
plt.subplot(2,2,1)
plt.imshow(np.dot(np.dot(C, Pi), C.transpose()) + np.diag(R), interpolation='none')
plt.colorbar()
plt.subplot(2,2,2)
Pi_true = np.array([sp.linalg.solve_discrete_lyapunov(trueParameters['A'], trueParameters['Q'])])[0,:,:]
plt.imshow(np.dot(np.dot(trueParameters['C'],Pi_true),trueParameters['C'].transpose()) +
           np.diag(trueParameters['R']),
           interpolation='none')
plt.colorbar()
plt.subplot(2,2,3)
plt.imshow(np.dot(np.dot(np.dot(C, A), Pi), C.transpose()), interpolation='none' )
plt.colorbar()
plt.subplot(2,2,4)
plt.imshow(np.dot(np.dot(np.dot(trueParameters['C'], trueParameters['A']), Pi_true), 
                                trueParameters['C'].transpose()), 
          interpolation='none' )
plt.colorbar()

plt.figure(2,figsize=(15,15))
plt.subplot(2,2,1)
plt.plot(d)
plt.plot(trueParameters['d'])
plt.legend(['true', 'est'])
plt.title('d')
plt.subplot(2,2,2)
plt.plot(R)
plt.plot(trueParameters['R'])
plt.legend(['true', 'est'])
plt.title('R')
plt.subplot(2,2,3)
plt.plot(np.sort(np.linalg.eig(A)[0]))
plt.plot(np.sort(np.diag(np.linalg.eig(trueParameters['A'])[0])))
plt.legend(['true', 'est'])
plt.title('eig(A)')
plt.subplot(2,2,4)


In [7]:
sfs = '/home/marcel/Desktop/Projects/Stitching/results/test_runs/LDS_setup_test'
npzloadfile = np.load(sf+'_tmp_'+str(41)+'.npz')
npzloadfile['arr_2']

array([[  5.50749899e-01,   6.21232002e-02,   3.12364327e-02,
         -3.32067298e-02,  -1.16858545e-02,   2.55868235e-02,
          1.34068021e-02,   6.79250392e-02,   4.23456909e-03,
          3.56323272e-02],
       [  3.98644631e-02,   7.61127790e-01,  -7.90067044e-02,
         -3.18583566e-02,  -2.54275809e-02,  -1.03823612e-01,
          6.32736829e-03,   5.10687347e-02,   4.38683138e-03,
          2.59130197e-02],
       [  2.80684223e-02,  -7.85257051e-02,   4.46354902e-01,
          1.39779162e-02,  -4.58212068e-02,   1.58524896e-02,
          4.92036036e-03,   1.68693522e-02,  -4.80054792e-03,
          2.38374621e-02],
       [ -1.62923999e-02,  -1.84139104e-02,   1.87833589e-03,
          3.56451968e-01,   3.61715015e-02,   7.63858119e-03,
         -2.41353236e-02,   2.91024838e-02,  -2.22470794e-02,
          2.04243087e-02],
       [  3.30635623e-04,   1.12885731e-02,  -2.89510582e-02,
         -3.25375859e-02,   8.42992180e-01,  -8.67301054e-02,
         -1.10056190e-02

In [ ]:
covy_h= np.dot(np.dot(C, Pi), C.transpose()) + np.diag(R)
covy_t=(np.dot(np.dot(trueParameters['C'],Pi_true),trueParameters['C'].transpose()) +
              np.diag(trueParameters['R']))

y_tl = np.zeros([2*yDim,T-1])
y_tl[range(yDim),:] = yOut[:,range(0,T-1),0]
y_tl[range(yDim,2*yDim),:] = yOut[:,range(1,T),0]
covy = np.cov(y_tl)

covy_e=   covy[np.ix_(range(yDim),range(yDim))]
covy_tl_e=covy[np.ix_(range(0,yDim),range(yDim,2*yDim))]

covy_tl_h= np.dot(np.dot(C, np.dot(A,Pi)), C.transpose())
covy_tl_t=(np.dot(np.dot(trueParameters['C'],np.dot(trueParameters['A'], Pi_true)),trueParameters['C'].transpose()))


In [ ]:
corry = np.corrcoef(yOut[:,:,0])
corry.shape

In [ ]:
%matplotlib qt
plt.imshow(corry, interpolation='none')
plt.colorbar()

In [ ]:
%matplotlib inline

plt.imshow(idxStitched,interpolation='none')

In [ ]:
%matplotlib inline
plt.figure(1, figsize=(20,10))
plt.subplot(1,3,1)
plt.plot(covy_e[np.invert(idxStitched)], covy_t[np.invert(idxStitched)], '.')
plt.title('emp vs. true')
plt.subplot(1,3,2)
plt.plot(covy_e[np.invert(idxStitched)], covy_h[np.invert(idxStitched)], '.')
plt.title('emp vs. stitched')
plt.subplot(1,3,3)
plt.plot(covy_t[np.invert(idxStitched)], covy_h[np.invert(idxStitched)], '.')
plt.title('true vs. stitched')

plt.figure(2, figsize=(20,10))
plt.subplot(1,3,1)
plt.plot(covy_tl_e[np.invert(idxStitched)], covy_tl_t[np.invert(idxStitched)], '.')
plt.title('emp vs. true')
plt.subplot(1,3,2)
plt.plot(covy_tl_e[np.invert(idxStitched)], covy_tl_h[np.invert(idxStitched)], '.')
plt.title('emp vs. stitched')
plt.subplot(1,3,3)
plt.plot(covy_tl_t[np.invert(idxStitched)], covy_tl_h[np.invert(idxStitched)], '.')
plt.title('true vs. stitched')

plt.figure(3, figsize=(20,10))
plt.subplot(1,3,1)
plt.plot(covy_e[idxStitched], covy_t[idxStitched], '.')
plt.title('emp vs. true')
plt.subplot(1,3,2)
plt.plot(covy_e[idxStitched], covy_h[idxStitched], '.')
plt.title('emp vs. stitched')
plt.subplot(1,3,3)
plt.plot(covy_t[idxStitched], covy_h[idxStitched], '.')
plt.title('true vs. stitched')

plt.figure(4, figsize=(20,10))
plt.subplot(1,3,1)
plt.plot(covy_tl_e[idxStitched], covy_tl_t[idxStitched], '.')
plt.title('emp vs. true')
plt.subplot(1,3,2)
plt.plot(covy_tl_e[idxStitched], covy_tl_h[idxStitched], '.')
plt.title('emp vs. stitched')
plt.subplot(1,3,3)
plt.plot(covy_tl_t[idxStitched], covy_tl_h[idxStitched], '.')
plt.title('true vs. stitched')